In [4]:
import json
import os
import re
import pandas as pd
import matplotlib as mpl
import numpy as np
import logging
import glob
import shutil
from distutils.dir_util import copy_tree
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from scipy.signal import savgol_filter

import glob
import pickle
import os
import logging
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp
pd.options.mode.chained_assignment = None

from pickle import load

import pandas as pd

In [2]:
def get_csv_files(dataset_path, keyword):
    """
    Generator function to recursively output the CSV files in a directory and its sub-directories.
    Arguments:
        dataset_path: Path to the directory containing the CSV files.
    Outputs:
        Paths of the found CSV files.
    """
    files = []
 
    for f in glob.glob(os.path.join(dataset_path,"*{}*".format(keyword))):
        #print(f)
        if not os.path.isdir(f):
            file_name, extension = f.split('.')            
            if extension == "csv":
                files.append(f)
            else:
                logging.warn("Invalid file: {}. Ignoring...".format(f))

    return files

In [3]:
def add_eulers(dataset_df):
    #df = pd.read_csv(dataset_path, index_col=None)
    df = dataset_df
    qs_pred = df[['qx', 'qy', 'qz', 'qw']]
    
    #print(qs_pred)
    
    rots_pred = R.from_quat(qs_pred) #needs xyzw input shape
    e_preds = rots_pred.as_euler('ZXY', degrees=True)
    
    #new_df = np.hstack((pos_pred_arr, e_preds)) --> hier später noch position adden
    e_df = pd.DataFrame(e_preds, columns=['pitch','roll', 'yaw'])
    
    result = pd.concat([df, e_df], axis=1, join="inner")
    
    return result

In [4]:
def load_df(path): 
    df = pd.read_csv(path, index_col=None)

    return df

In [3]:
def load_scaler_objects_rotation(path_scaler_objects):
    
    out_seq4_scaler = load(open(path_scaler_objects + 'in_seq4_scaler.pkl', 'rb'))
    out_seq5_scaler = load(open(path_scaler_objects + 'in_seq5_scaler.pkl', 'rb'))
    out_seq6_scaler = load(open(path_scaler_objects + 'in_seq6_scaler.pkl', 'rb'))
    out_seq7_scaler = load(open(path_scaler_objects + 'in_seq7_scaler.pkl', 'rb'))
    
    return out_seq4_scaler, out_seq5_scaler, out_seq6_scaler, out_seq7_scaler

In [7]:
def load_scaler_objects_euler(path_scaler_objects):
    
    out_seq4_scaler = load(open(path_scaler_objects + 'in_seq4_scaler.pkl', 'rb'))
    out_seq5_scaler = load(open(path_scaler_objects + 'in_seq5_scaler.pkl', 'rb'))
    out_seq6_scaler = load(open(path_scaler_objects + 'in_seq6_scaler.pkl', 'rb'))
    
    return out_seq4_scaler, out_seq5_scaler, out_seq6_scaler